In [ ]:
!pip install roboflow google-cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.9/74.9 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled idna-3.7
  Attempting uninstall: cycler
    Found existing installation: cycler 0.12.1
    Uninstalling cycler-0.12.1:
      Successfully uninstalled cycler-0.12

In [ ]:
import cv2
from roboflow import Roboflow
import os
import time

# Initialize Roboflow model
rf = Roboflow(api_key="")
project = rf.workspace().project("publicnotice")
model = project.version("2").model

# Define the folder containing images
folder_path = "/content/newspaper/"
count = 0
os.makedirs("notice", exist_ok=True)
for filename in os.listdir(folder_path):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        thresh_inv = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        blur = cv2.GaussianBlur(thresh_inv, (1, 1), 0)
        thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]

        for i, c in enumerate(contours):
            x, y, w, h = cv2.boundingRect(c)
            if w >= 450 and h >= 350:
                roi = img[y:y+h, x:x+w]
                temp_filename = f"temp_roi_{i}.jpg"
                cv2.imwrite(temp_filename, roi)

                prediction = model.predict(temp_filename, confidence=85, overlap=30).json()

                if len(prediction['predictions']) > 0:
                    timestamp = int(time.time())
                    cv2.imwrite(f"notice/notice_{timestamp}.jpg", roi)

        # Clean up temporary files
        for temp_file in os.listdir():
            if temp_file.startswith("temp_roi_"):
                os.remove(temp_file)


loading Roboflow workspace...
loading Roboflow project...


***Store notice images in Cloud Storage***

In [ ]:
from google.cloud import storage,firestore
import os
from datetime import date
from google.oauth2 import service_account
from google.colab import drive

drive.mount('/content/drive')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = ""
credentials = service_account.Credentials.from_service_account_file('')

db = firestore.Client(credentials=credentials)

def upload_images_to_gcs(images_folder):
    storage_client = storage.Client()

    bucket_name = "xyz.appspot.com"

    bucket = storage_client.bucket(bucket_name)

    today_date = str(date.today())

    folder_blob = bucket.blob("NewspaperNotice/")

    folder_blob.upload_from_string("")

    for filename in os.listdir(images_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            blob = bucket.blob("NewspaperNotice/" + filename)
            blob.upload_from_filename(os.path.join(images_folder, filename))


            blob.make_public()

            # Use the public URL
            public_url = blob.public_url
            print(public_url)
            doc_ref = db.collection(u'publicNotice').document()
            doc_ref.set({
                'URL': public_url,
                'date': today_date,
                'newspaper':"Gujarat Samachar"

            })

            os.remove(os.path.join(images_folder, filename))

images_folder = "/content/notice"
upload_images_to_gcs(images_folder)

Mounted at /content/drive
https://storage.googleapis.com/townplanmap.appspot.com/NewspaperNotice/notice_1716353923.jpg
https://storage.googleapis.com/townplanmap.appspot.com/NewspaperNotice/notice_1716353898.jpg
https://storage.googleapis.com/townplanmap.appspot.com/NewspaperNotice/notice_1716353921.jpg
https://storage.googleapis.com/townplanmap.appspot.com/NewspaperNotice/notice_1716353907.jpg
https://storage.googleapis.com/townplanmap.appspot.com/NewspaperNotice/notice_1716353916.jpg
https://storage.googleapis.com/townplanmap.appspot.com/NewspaperNotice/notice_1716353924.jpg
https://storage.googleapis.com/townplanmap.appspot.com/NewspaperNotice/notice_1716353900.jpg
https://storage.googleapis.com/townplanmap.appspot.com/NewspaperNotice/notice_1716353897.jpg
https://storage.googleapis.com/townplanmap.appspot.com/NewspaperNotice/notice_1716353925.jpg
https://storage.googleapis.com/townplanmap.appspot.com/NewspaperNotice/notice_1716353917.jpg
https://storage.googleapis.com/townplanmap.a

***Store Notice Papers in Google Cloud***

In [ ]:
from google.cloud import storage
import os
from datetime import date
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/xyz.json"

def upload_images_to_gcs(images_folder):
    # Initialize Google Cloud Storage client
    storage_client = storage.Client()

    # Define your Google Cloud Storage bucket name
    bucket_name = ""

    # Access the bucket
    bucket = storage_client.bucket(bucket_name)

    # Get today's date in the format YYYY-MM-DD
    today_date = str(date.today())

    # Create a blob object for the new folder
    folder_blob = bucket.blob(today_date + "_newspaper/")

    # Create the folder by uploading an empty file with a trailing slas
    folder_blob.upload_from_string("")

    # Process each image in the specified folder and upload to the newly created folder in GCS
    for filename in os.listdir(images_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            # Create a blob object for the image
            blob = bucket.blob(today_date + "_newspaper/" + filename)
            blob.upload_from_filename(os.path.join(images_folder, filename))

            # Optionally, delete the local file after uploading
            os.remove(os.path.join(images_folder, filename))

# Call the function to upload images to Google Cloud Storage
images_folder = "/content/newspaper"
upload_images_to_gcs(images_folder)

DefaultCredentialsError: File /content/townplanmap-fc67b1ca73e0.json was not found.

***Key Added to database***

In [ ]:
from google.colab import drive
drive.mount('')

Mounted at /content/drive


In [ ]:
from google.cloud import storage,firestore
import os
from datetime import date
from google.oauth2 import service_account
from google.colab import drive
drive.mount('/content/drive')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = ""
credentials = service_account.Credentials.from_service_account_file('')
db = firestore.Client(credentials=credentials)
collection_ref = db.collection("publicNotice")
docs = collection_ref.stream()

# Iterate over each document and update the specified field
for doc in docs:
    doc_ref = collection_ref.document(doc.id)
    doc_ref.update({'city':'auda'})
    print(f"Document with ID {doc.id} updated successfully.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Document with ID 0zjHsxvHaVnBcaQHzOxp updated successfully.
Document with ID 1CZiEuKRqSgnIvxQstMu updated successfully.
Document with ID 1j1rIaocE9G5oJHYGQXv updated successfully.
Document with ID 1jTFhyWdrFOF5oumFh7q updated successfully.
Document with ID 2Hta3olLuLk2cByaUDW2 updated successfully.
Document with ID 2gHXxk2KOecDZhxNxcfa updated successfully.
Document with ID 2lfsZvFgYIkGcxhcgX5r updated successfully.
Document with ID 2xYAvLgER1wVSFnA6lSR updated successfully.
Document with ID 30swYWNJIzbafxqvnzfl updated successfully.
Document with ID 320GT6WOQhduVPdue3qX updated successfully.
Document with ID 355zwtDEwDdeD2cdteb1 updated successfully.
Document with ID 39i6sd636ok7eoWNICLn updated successfully.
Document with ID 3vbPp80jlqhcKgT67Wlt updated successfully.
Document with ID 4G2HcziEKWBf7rJ0z4KD updated successfully.
Document with ID 4PdCy8g5RWEfMo

In [ ]:
!pip install pytesseract
!sudo apt install tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (6,158 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [ ]:
from PIL import Image
import pytesseract

image = Image.open("/content/notice_1716268460.jpg")

In [ ]:
import pytesseract
import shutil
import os
import random
try:
    from PIL import Image
except ImportError:
    import Image

In [ ]:
text = pytesseract.image_to_string(image, lang='guj')

In [ ]:
print(text)

જાહેર નોટિસ

આથી જાહેર જનતાને જણાવવામાં
આવે છે કે, ડિસ્ટ્રીકટ: અમદાવાદ,
તાલુકા : સાબરમતીના મોજે ગામ :
વાસણાની સીમમાં આવેલ સર્વે નંબર
૨૫૩ પૈકી ની ક્ષેતફળ ૯૧૧
ચો.મી.ની જમીન કે જેનો સમાવેશ
ટાઉન પ્લાનીંગ સ્કીમ નંબર

ર૬(વાસણા) માં થતા તે બદલ
ફાળવવામાં આવેલ ફાઈનલ પ્લોટ
નંબર ૭૭ ની ક્ષેત્રફળ ૭૯૭

ચો.મી.ની જમીન કે જેનો નવો સર્વે
નંબર 02677 ફાળવવામાં આવેલ
તથા તેનો સીટી સર્વેમાં સમાવેશ થતા
તે બદલ ફાળવવામાં આવેલ સીટી સર્વે
નંબર NA0t26f77 ની ક્ષેત્રફળ ૭૯૭
ચો.મી.ની જમીન (૧) હરેશભાઈ
પુંજાભાઈ પટેલ તથા (૨) પિયુષભાઈ
પુંજાભાઈ પટેલ નાઓના માલીકીપણા
બાબત, અમારા અસીલ દ્વારા અમારી
સમક્ષ ટાઈટલ કલીયર સર્ટીફીકેટની
માંગણી કરેલ છે. સબબ સર્વે શખ્સો
કે જેમનો સદરહુ જમીનમાં ખોરાકી,

 

પોષાકી, કોઈપણ પ્રકારનો લાગભાગ,
હક્ક, હિસ્સો, ચાર્જ, બોજો અલાખો કે
લિયન ચાર્જ પોષાતો હોય તો તેઓને
આ નોટીસ પ્રસિ થયેથી દિન-૭ (સાત)
માં તે અંગેના તમામ લેખિત પુરાવા
સહીત નીચેના સરનામે રજીસ્ટર્ડ ટપાલ
દ્વારા જાણ કરી વાંધો નોંધાવવો. જો તેમ
કરવામાં નહિ આવેતો સદરહુ જમીનમાં
કોઈનો કોઈપણ પ્રકારનો હક્ક, હિસ્સો
પોષાતો હોય તો તે જતો યાને કે “વેઈવ'
કરે

In [ ]:
!ls /usr/share/tesseract-ocr/4.00/tessdata

configs  eng.traineddata  osd.traineddata  pdf.ttf  tessconfigs


In [ ]:
!cp /content/guj.traineddata /usr/share/tesseract-ocr/4.00/tessdata/

In [ ]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.30.1
    Uninstalling openai-1.30.1:
      Successfully uninstalled openai-1.30.1


In [ ]:
import requests

# Define the API endpoint and your API key
api_endpoint = "https://api.example.com/v1/analyze"
api_key = ""

# Define the headers, including the API key for authentication
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# Define the data you want to send in the request
data = {
    "text": """જાહેર નોટિસ આથી જાહેર જનતાને જણાવવામાં આવે છે કે, ડિસ્ટ્રીકટ: અમદાવાદ, તાલુકા : સાબરમતીના મોજે ગામ :
        વાસણાની સીમમાં આવેલ સર્વે નંબર
        ૨૫૩ પૈકી ની ક્ષેતફળ ૯૧૧
        ચો.મી.ની જમીન કે જેનો સમાવેશ
        ટાઉન પ્લાનીંગ સ્કીમ નંબર

        ર૬(વાસણા) માં થતા તે બદલ
        ફાળવવામાં આવેલ ફાઈનલ પ્લોટ
        નંબર ૭૭ ની ક્ષેત્રફળ ૭૯૭

        ચો.મી.ની જમીન કે જેનો નવો સર્વે
        નંબર 02677 ફાળવવામાં આવેલ
        તથા તેનો સીટી સર્વેમાં સમાવેશ થતા
        તે બદલ ફાળવવામાં આવેલ સીટી સર્વે
        નંબર NA0t26f77 ની ક્ષેત્રફળ ૭૯૭
        ચો.મી.ની જમીન (૧) હરેશભાઈ
        પુંજાભાઈ પટેલ તથા (૨) પિયુષભાઈ
        પુંજાભાઈ પટેલ નાઓના માલીકીપણા
        બાબત, અમારા અસીલ દ્વારા અમારી
        સમક્ષ ટાઈટલ કલીયર સર્ટીફીકેટની
        માંગણી કરેલ છે. સબબ સર્વે શખ્સો
        કે જેમનો સદરહુ જમીનમાં ખોરાકી,

        પોષાકી, કોઈપણ પ્રકારનો લાગભાગ,
        હક્ક, હિસ્સો, ચાર્જ, બોજો અલાખો કે
        લિયન ચાર્જ પોષાતો હોય તો તેઓને
        આ નોટીસ પ્રસિ થયેથી દિન-૭ (સાત)
        માં તે અંગેના તમામ લેખિત પુરાવા
        સહીત નીચેના સરનામે રજીસ્ટર્ડ ટપાલ
        દ્વારા જાણ કરી વાંધો નોંધાવવો. જો તેમ
        કરવામાં નહિ આવેતો સદરહુ જમીનમાં
        કોઈનો કોઈપણ પ્રકારનો હક્ક, હિસ્સો
        પોષાતો હોય તો તે જતો યાને કે “વેઈવ'
        કરેલ છે તેમ સમજી મુદત વિત્યે સદરહુ
        જમીનનું ટાઈટલ કલીયર સર્ટીફીકેટ ઈસ્યુ
        કરવામાં આવશે અતે ત્યારબાદ કોઈની
        કોઈપણ પ્રકારની તકરાર માન્ય રખાશે
        નહિ તે જાણવું.
        શ્રીનાથ પરીખ
        વાડીયા ઘાંડી એન્ડ ક.
        (અમદાવાદ)
        (એડવોકેટસ એન્ડ સોલીસીટર્સ)
        ૧ લો માળ, ચંદન હાઉસ, મેયરના
        બંગલાની બાજુમાં, લો-ગાર્ડન,
        અમદાવાદ- ૩૮૦૦૦૬
        ફોન : (૦૭૯) ૨૬૫૬૪૭૦૦
        ૨૬૫૬૪૮૦૦

        Give a name of District, Taluka, Village, Surveynumber in json format.
        """


}


# Make the request to the API
response = requests.post(api_endpoint, headers=headers, json=data)

# Check the response status and print the result
if response.status_code == 200:
    result = response.json()
    print("Response:", result)
else:
    print("Error:", response.status_code, response.text)


ConnectionError: HTTPSConnectionPool(host='api.example.com', port=443): Max retries exceeded with url: /v1/analyze (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7e8c8c0d3160>: Failed to resolve 'api.example.com' ([Errno -2] Name or service not known)"))

In [ ]:
import openai

# Define your OpenAI API key
openai.api_key = ""

# Define the large block of text
large_text = """ જાહેર નોટિસ આથી જાહેર જનતાને જણાવવામાં આવે છે કે, ડિસ્ટ્રીકટ: અમદાવાદ, તાલુકા : સાબરમતીના મોજે ગામ :
        વાસણાની સીમમાં આવેલ સર્વે નંબર
        ૨૫૩ પૈકી ની ક્ષેતફળ ૯૧૧
        ચો.મી.ની જમીન કે જેનો સમાવેશ
        ટાઉન પ્લાનીંગ સ્કીમ નંબર

        ર૬(વાસણા) માં થતા તે બદલ
        ફાળવવામાં આવેલ ફાઈનલ પ્લોટ
        નંબર ૭૭ ની ક્ષેત્રફળ ૭૯૭

        ચો.મી.ની જમીન કે જેનો નવો સર્વે
        નંબર 02677 ફાળવવામાં આવેલ
        તથા તેનો સીટી સર્વેમાં સમાવેશ થતા
        તે બદલ ફાળવવામાં આવેલ સીટી સર્વે
        નંબર NA0t26f77 ની ક્ષેત્રફળ ૭૯૭
        ચો.મી.ની જમીન (૧) હરેશભાઈ
        પુંજાભાઈ પટેલ તથા (૨) પિયુષભાઈ
        પુંજાભાઈ પટેલ નાઓના માલીકીપણા
        બાબત, અમારા અસીલ દ્વારા અમારી
        સમક્ષ ટાઈટલ કલીયર સર્ટીફીકેટની
        માંગણી કરેલ છે. સબબ સર્વે શખ્સો
        કે જેમનો સદરહુ જમીનમાં ખોરાકી,

        પોષાકી, કોઈપણ પ્રકારનો લાગભાગ,
        હક્ક, હિસ્સો, ચાર્જ, બોજો અલાખો કે
        લિયન ચાર્જ પોષાતો હોય તો તેઓને
        આ નોટીસ પ્રસિ થયેથી દિન-૭ (સાત)
        માં તે અંગેના તમામ લેખિત પુરાવા
        સહીત નીચેના સરનામે રજીસ્ટર્ડ ટપાલ
        દ્વારા જાણ કરી વાંધો નોંધાવવો. જો તેમ
        કરવામાં નહિ આવેતો સદરહુ જમીનમાં
        કોઈનો કોઈપણ પ્રકારનો હક્ક, હિસ્સો
        પોષાતો હોય તો તે જતો યાને કે “વેઈવ'
        કરેલ છે તેમ સમજી મુદત વિત્યે સદરહુ
        જમીનનું ટાઈટલ કલીયર સર્ટીફીકેટ ઈસ્યુ
        કરવામાં આવશે અતે ત્યારબાદ કોઈની
        કોઈપણ પ્રકારની તકરાર માન્ય રખાશે
        નહિ તે જાણવું.
        શ્રીનાથ પરીખ
        વાડીયા ઘાંડી એન્ડ ક.
        (અમદાવાદ)
        (એડવોકેટસ એન્ડ સોલીસીટર્સ)
        ૧ લો માળ, ચંદન હાઉસ, મેયરના
        બંગલાની બાજુમાં, લો-ગાર્ડન,
        અમદાવાદ- ૩૮૦૦૦૬
        ફોન : (૦૭૯) ૨૬૫૬૪૭૦૦
        ૨૬૫૬૪૮૦૦

        Give a name of District, Taluka, Village, Surveynumber in json format.
        eg. {
          District:"x",
          Taluka:"y",
          Village:"z",
          surveynumber:"w"
        }

"""

# Make the request to the OpenAI API
response = openai.Completion.create(
    engine="gpt-3.5-turbo",
    prompt=large_text,
    max_tokens=100,
    n=1,
    stop=None,
    temperature=0.7
)

# Print the result
print(response.choices[0].text.strip())

APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import os
os.environ['OPENAI_API_KEY'] = ''
client = OpenAI()

large_text = """ જાહેર નોટિસ આથી જાહેર જનતાને જણાવવામાં આવે છે કે, ડિસ્ટ્રીકટ: અમદાવાદ, તાલુકા : સાબરમતીના મોજે ગામ :
        વાસણાની સીમમાં આવેલ સર્વે નંબર
        ૨૫૩ પૈકી ની ક્ષેતફળ ૯૧૧
        ચો.મી.ની જમીન કે જેનો સમાવેશ
        ટાઉન પ્લાનીંગ સ્કીમ નંબર

        ર૬(વાસણા) માં થતા તે બદલ
        ફાળવવામાં આવેલ ફાઈનલ પ્લોટ
        નંબર ૭૭ ની ક્ષેત્રફળ ૭૯૭

        ચો.મી.ની જમીન કે જેનો નવો સર્વે
        નંબર 02677 ફાળવવામાં આવેલ
        તથા તેનો સીટી સર્વેમાં સમાવેશ થતા
        તે બદલ ફાળવવામાં આવેલ સીટી સર્વે
        નંબર NA0t26f77 ની ક્ષેત્રફળ ૭૯૭
        ચો.મી.ની જમીન (૧) હરેશભાઈ
        પુંજાભાઈ પટેલ તથા (૨) પિયુષભાઈ
        પુંજાભાઈ પટેલ નાઓના માલીકીપણા
        બાબત, અમારા અસીલ દ્વારા અમારી
        સમક્ષ ટાઈટલ કલીયર સર્ટીફીકેટની
        માંગણી કરેલ છે. સબબ સર્વે શખ્સો
        કે જેમનો સદરહુ જમીનમાં ખોરાકી,

        પોષાકી, કોઈપણ પ્રકારનો લાગભાગ,
        હક્ક, હિસ્સો, ચાર્જ, બોજો અલાખો કે
        લિયન ચાર્જ પોષાતો હોય તો તેઓને
        આ નોટીસ પ્રસિ થયેથી દિન-૭ (સાત)
        માં તે અંગેના તમામ લેખિત પુરાવા
        સહીત નીચેના સરનામે રજીસ્ટર્ડ ટપાલ
        દ્વારા જાણ કરી વાંધો નોંધાવવો. જો તેમ
        કરવામાં નહિ આવેતો સદરહુ જમીનમાં
        કોઈનો કોઈપણ પ્રકારનો હક્ક, હિસ્સો
        પોષાતો હોય તો તે જતો યાને કે “વેઈવ'
        કરેલ છે તેમ સમજી મુદત વિત્યે સદરહુ
        જમીનનું ટાઈટલ કલીયર સર્ટીફીકેટ ઈસ્યુ
        કરવામાં આવશે અતે ત્યારબાદ કોઈની
        કોઈપણ પ્રકારની તકરાર માન્ય રખાશે
        નહિ તે જાણવું.
        શ્રીનાથ પરીખ
        વાડીયા ઘાંડી એન્ડ ક.
        (અમદાવાદ)
        (એડવોકેટસ એન્ડ સોલીસીટર્સ)
        ૧ લો માળ, ચંદન હાઉસ, મેયરના
        બંગલાની બાજુમાં, લો-ગાર્ડન,
        અમદાવાદ- ૩૮૦૦૦૬
        ફોન : (૦૭૯) ૨૬૫૬૪૭૦૦
        ૨૬૫૬૪૮૦૦

        Give a name of District, Taluka, Village, Surveynumber in json format.
        eg. {
          District:"x",
          Taluka:"y",
          Village:"z",
          surveynumber:"w"
        }

"""

response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": large_text}
  ]
)
print(response.choices[0].message.content)

{
  "District": "અમદાવાદ",
  "Taluka": "સાબરમતીના મોજે",
  "Village": "વાસણા",
  "surveynumber": "૨૫૩"
}


In [ ]:
import openai
openai.api_key = ""

prompt = "Hello, my name is John and I am a software engineer."
large_text = """ જાહેર નોટિસ આથી જાહેર જનતાને જણાવવામાં આવે છે કે, ડિસ્ટ્રીકટ: અમદાવાદ, તાલુકા : સાબરમતીના મોજે ગામ :
        વાસણાની સીમમાં આવેલ સર્વે નંબર
        ૨૫૩ પૈકી ની ક્ષેતફળ ૯૧૧
        ચો.મી.ની જમીન કે જેનો સમાવેશ
        ટાઉન પ્લાનીંગ સ્કીમ નંબર

        ર૬(વાસણા) માં થતા તે બદલ
        ફાળવવામાં આવેલ ફાઈનલ પ્લોટ
        નંબર ૭૭ ની ક્ષેત્રફળ ૭૯૭

        ચો.મી.ની જમીન કે જેનો નવો સર્વે
        નંબર 02677 ફાળવવામાં આવેલ
        તથા તેનો સીટી સર્વેમાં સમાવેશ થતા
        તે બદલ ફાળવવામાં આવેલ સીટી સર્વે
        નંબર NA0t26f77 ની ક્ષેત્રફળ ૭૯૭
        ચો.મી.ની જમીન (૧) હરેશભાઈ
        પુંજાભાઈ પટેલ તથા (૨) પિયુષભાઈ
        પુંજાભાઈ પટેલ નાઓના માલીકીપણા
        બાબત, અમારા અસીલ દ્વારા અમારી
        સમક્ષ ટાઈટલ કલીયર સર્ટીફીકેટની
        માંગણી કરેલ છે. સબબ સર્વે શખ્સો
        કે જેમનો સદરહુ જમીનમાં ખોરાકી,

        પોષાકી, કોઈપણ પ્રકારનો લાગભાગ,
        હક્ક, હિસ્સો, ચાર્જ, બોજો અલાખો કે
        લિયન ચાર્જ પોષાતો હોય તો તેઓને
        આ નોટીસ પ્રસિ થયેથી દિન-૭ (સાત)
        માં તે અંગેના તમામ લેખિત પુરાવા
        સહીત નીચેના સરનામે રજીસ્ટર્ડ ટપાલ
        દ્વારા જાણ કરી વાંધો નોંધાવવો. જો તેમ
        કરવામાં નહિ આવેતો સદરહુ જમીનમાં
        કોઈનો કોઈપણ પ્રકારનો હક્ક, હિસ્સો
        પોષાતો હોય તો તે જતો યાને કે “વેઈવ'
        કરેલ છે તેમ સમજી મુદત વિત્યે સદરહુ
        જમીનનું ટાઈટલ કલીયર સર્ટીફીકેટ ઈસ્યુ
        કરવામાં આવશે અતે ત્યારબાદ કોઈની
        કોઈપણ પ્રકારની તકરાર માન્ય રખાશે
        નહિ તે જાણવું.
        શ્રીનાથ પરીખ
        વાડીયા ઘાંડી એન્ડ ક.
        (અમદાવાદ)
        (એડવોકેટસ એન્ડ સોલીસીટર્સ)
        ૧ લો માળ, ચંદન હાઉસ, મેયરના
        બંગલાની બાજુમાં, લો-ગાર્ડન,
        અમદાવાદ- ૩૮૦૦૦૬
        ફોન : (૦૭૯) ૨૬૫૬૪૭૦૦
        ૨૬૫૬૪૮૦૦
"""
# Make the request to the OpenAI API using the chat completion method
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": """You need give the data of District, Taluka, Village, Surveynumber in json format. {District:"x",Taluka:"y",Village:"z",surveynumber:"w"}"""},
        {"role": "user", "content": large_text}
    ],
    max_tokens=200  # Adjust as needed
)

# Extract and print the generated text
generated_text = response.choices[0].message['content'].strip()
print(generated_text)

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
import openai

# Define your OpenAI API key

# Define the large block of text
text = """
જાહેર નોટિસ આથી જાહેર જનતાને જણાવવામાં આવે છે કે, ડિસ્ટ્રીકટ: અમદાવાદ, તાલુકા : સાબરમતીના મોજે ગામ :
વાસણાની સીમમાં આવેલ સર્વે નંબર
૨૫૩ પૈકી ની ક્ષેતફળ ૯૧૧
ચો.મી.ની જમીન કે જેનો સમાવેશ
ટાઉન પ્લાનીંગ સ્કીમ નંબર

ર૬(વાસણા) માં થતા તે બદલ
ફાળવવામાં આવેલ ફાઈનલ પ્લોટ
નંબર ૭૭ ની ક્ષેત્રફળ ૭૯૭

ચો.મી.ની જમીન કે જેનો નવો સર્વે
નંબર 02677 ફાળવવામાં આવેલ
તથા તેનો સીટી સર્વેમાં સમાવેશ થતા
તે બદલ ફાળવવામાં આવેલ સીટી સર્વે
નંબર NA0t26f77 ની ક્ષેત્રફળ ૭૯૭
ચો.મી.ની જમીન (૧) હરેશભાઈ
પુંજાભાઈ પટેલ તથા (૨) પિયુષભાઈ
પુંજાભાઈ પટેલ નાઓના માલીકીપણા
બાબત, અમારા અસીલ દ્વારા અમારી
સમક્ષ ટાઈટલ કલીયર સર્ટીફીકેટની
માગણી કરેલ છે. સબબ સર્વે શખ્સો
કે જેમનો સદરહુ જમીનમાં ખોરાકી,

પોષાકી, કોઈપણ પ્રકારનો લાગભાગ,
હક્ક, હિસ્સો, ચાર્જ, બોજો અલાખો કે
લિયન ચાર્જ પોષાતો હોય તો તેઓને
આ નોટીસ પ્રસિ થયેથી દિન-૭ (સાત)
માં તે અંગેના તમામ લેખિત પુરાવા
સહીત નીચેના સરનામે રજીસ્ટર્ડ ટપાલ
દ્વારા જાણ કરી વાંધો નોંધાવવો. જો તેમ
કરવામાં નહિ આવેતો સદરહુ જમીનમાં
કોઈનો કોઈપણ પ્રકારનો હક્ક, હિસ્સો
પોષાતો હોય તો તે જતો યાને કે “વેઈવ'
કરેલ છે તેમ સમજી મુદત વિત્યે સદરહુ
જમીનનું ટાઈટલ કલીયર સર્ટીફીકેટ ઈસ્યુ
કરવામાં આવશે અતે ત્યારબાદ કોઈની
કોઈપણ પ્રકારની તકરાર માન્ય રખાશે
નહીં તે જાણવું.
શ્રીનાથ પરીખ
વાડીયા ઘાંડી એન્ડ ક.
(અમદાવાદ)
(એડવોકેટસ એન્ડ સોલીસીટર્સ)
૧ લો માળ, ચંદન હાઉસ, મેયરના
બંગલાની બાજુમાં, લો-ગાર્ડન,
અમદાવાદ- ૩૮૦૦૦૬
ફોન : (૦૭૯) ૨૬૫૬૪૭૦૦
૨૬૫૬૪૮૦૦
"""

# Make the request to the OpenAI API using the chat completion method
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You need to give the data of District, Taluka, Village, Surveynumber in JSON format. {district: 'x', taluka: 'y', village: 'z', survey: 'w'}"},
        {"role": "user", "content": text}
    ],
    max_tokens=200  # Adjust as needed
)

# Extract and print the generated text
generated_text = response['choices'][0]['message']['content'].strip()
print(generated_text)


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
!openai --version

openai 1.30.1


In [ ]:
!pip install --upgrade openai

  Using cached openai-1.30.1-py3-none-any.whl (320 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [ ]:
from google.cloud import storage,firestore
import os
from datetime import date
from google.oauth2 import service_account
from google.colab import drive
drive.mount('/content/drive')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = ""
credentials = service_account.Credentials.from_service_account_file('')
db = firestore.Client(credentials=credentials)
collection_ref = db.collection("anyror")
docs = collection_ref.stream()

# Iterate over each document and update the specified field
for doc in docs:
    doc_ref = collection_ref.document(doc.id)
    doc_ref.update({'linkAdded':True})
    print(f"Document with ID {doc.id} updated successfully.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Document with ID 0Dob1RsMRkiSstw1sNl6 updated successfully.
Document with ID 0JaaRkvw7o2zyYZqc7AQ updated successfully.
Document with ID 0lfEQ727UKhq7Shnuudk updated successfully.
Document with ID 1Fk1hjc37HyLJD4AN7kY updated successfully.
Document with ID 1RSmmcg5pepxlWmGYfOa updated successfully.
Document with ID 2T515k720Bmc1jK2CaOS updated successfully.
Document with ID 2mGvM5XBesg7jIh1si6C updated successfully.
Document with ID 2vZl3nhQ0JclOisWFoCM updated successfully.
Document with ID 2xmrRjvt8KopJ1F5Wrie updated successfully.
Document with ID 35DYMrUtX6WPztS3w9Ig updated successfully.
Document with ID 3CycFfykhlLfzTieO3sG updated successfully.
Document with ID 3IoRh7NZO7lChPCdrtAU updated successfully.
Document with ID 3KzhxWzfZx6GZnkLaAui updated successfully.
Document with ID 3V1Ip23UmWiViH5I4YqZ updated successfully.
Document with ID 3VQNUt3S2YeDYe